In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import deque
import random

import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.Certificate('./serviceAccountKey.json')
default_app = firebase_admin.initialize_app(cred)
db = firestore.client()
print('Firebase Connected')

Firebase Connected


In [3]:
# install category_encoders library
!pip install category_encoders

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import category_encoders as ce

     |████████████████████████████████| 81kB 5.8MB/s 


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
# Collect user data from Firestore and put into DataFrame

users_docs = db.collection(u'users').stream()
user_key = []
user_list = []
for u in users_docs:
  user_key.append(list(u.to_dict().keys()))
  user_list.append(u.to_dict())

user_long_col = max(user_key, key=len)
user_df = pd.DataFrame(columns=user_long_col)
for doc in user_list:
  user_df = user_df.append(user_df.from_records([doc])).reset_index(drop=True)
  
user_df

,dobYear,userid,fname,dob,gender,age,email,lname,role,videoswatched,satisfaction
0,2021,0KAAFtAXIHYEG6WnQ5cq96a4ilO2,test6,2021 05 26,male,0,test6@testmail.com,test6,user,5,3.67
1,2006,2cBjJaRQKjWIjUi5oz1hTxZWZ0b2,gate1001,2006 11 21,male,15,gate1001@testmail.com,test,user,1,0.00
2,2021,A0SmGqVjMSbpclvoffQFqD7kB2G3,Norwati,2021 07 13,female,0,norwati@upm.edu.my,Mustapha,user,3,3.00
3,2000,Bu4A24oqfURu892PfkI1gwdzUit1,gate,2000 06 06,male,21,gatetest2021@test.com,test2021,male,NaN,NaN
4,2021,EWYk6VItyvflM9UgpqYVfhDddrE3,yk21,2021 06 22,male,0,yk21@test.com.my,test,user,NaN,NaN
5,2006,FXF6bjg9JLfELtuWl1XqYE7wDaE2,gate1002,2006 11 21,female,15,gate1002@testmail.com,test,user,1,0.00
6,2000,FdU1cWctC1dX7vNVbTzemBWp01I2,gate,2000 10 10,female,21,gatetest2020@test.com,test2020,admin,100,4.00
7,2000,H4qDaIws1cao4bKIr5w7KnD2c152,yk,2000 05 05,male,21,gatetest3@gmail.com,Ee,user,NaN,NaN
8,2021,MBz38WGDolMefm0EycloLTDbFsJ3,erzam,2021 02 12,male,0,erzam@upm.edu.my,marlisah,user,3,5.00
9,1972,RQi6TJZfVtNZqqNzXPt9S5eKqWx1,Razali,1972 05 06,male,49,razaliy@upm.edu.my,Yaakob,user,3,3.50


In [5]:
# Collect video data from Firestore and put into DataFrame

videos_docs = db.collection(u'videos').stream()
video_key = []
video_list = []
for v in videos_docs:
  video_key.append(list(v.to_dict().keys()))
  video_list.append(v.to_dict())

video_long_col = max(video_key, key=len)
video_df = pd.DataFrame(columns=video_long_col)
for doc in video_list:
  video_df = video_df.append(video_df.from_records([doc])).reset_index(drop=True)
  
video_df

,avgRating,videoId,uniqueviewers,view,category,youtubeId,title
0,3.42,2dZiMBwX_5Q,7,139,java,2dZiMBwX_5Q,Java Programming Tutorial 1 - Introduction to ...
1,4.5,2ePf9rue1Ao,3,12,artificial intelligence,2ePf9rue1Ao,What is Artificial Intelligence? In 5 minutes.
2,3,BvJYXl2ywUE,1,5,html,BvJYXl2ywUE,Introduction to HTML || Your First Web Page ||...
3,3.33,Hjl6gbg9kmk,1,15,html,Hjl6gbg9kmk,Web Development: Intro to HTML
4,3,Ig1nfPjrETc,NaN,3,machine learning,Ig1nfPjrETc,Machine Learning Tutorial: Supervised Learning
5,3,L--IxUH4fac,1,4,evolutionary algorithm,L--IxUH4fac,Evolutionary Algorithms
6,3.33,OXWvrRLzEaU,1,10,generative adversarial networks,OXWvrRLzEaU,An Introduction to Generative Adversarial Netw...
7,5,RaOejcczPas,NaN,3,digital twin,RaOejcczPas,"Introduction to Digital Twin: Simple, but deta..."
8,3.75,TlB_eWDSMt4,6,34,nodejs,TlB_eWDSMt4,Node.js Tutorial for Beginners: Learn Node in ...
9,5,USjZcfj8yxE,1,3,git,USjZcfj8yxE,Learn Git In 15 Minutes


In [6]:
# Collect rating data from Firestore and put into DataFrame

ratings_docs = db.collection(u'ratings').stream()
rating_key = []
rating_list = []
for u in ratings_docs:
  rating_key.append(list(u.to_dict().keys()))
  rating_list.append(u.to_dict())

rating_long_col = max(rating_key, key=len)
rating_df = pd.DataFrame(columns=rating_long_col)
for doc in rating_list:
  rating_df = rating_df.append(rating_df.from_records([doc])).reset_index(drop=True)
  
rating_df

,userId,videoId,value,whenRated
0,0KAAFtAXIHYEG6WnQ5cq96a4ilO2,2dZiMBwX_5Q,3,2021-05-31 16:00:00+00:00
1,0KAAFtAXIHYEG6WnQ5cq96a4ilO2,Hjl6gbg9kmk,5,NaT
2,0KAAFtAXIHYEG6WnQ5cq96a4ilO2,OXWvrRLzEaU,3,NaT
3,0KAAFtAXIHYEG6WnQ5cq96a4ilO2,TlB_eWDSMt4,3,2021-06-21 04:24:05.908000+00:00
4,0KAAFtAXIHYEG6WnQ5cq96a4ilO2,W6NZfCO5SIk,4,NaT
...,...,...,...,...
56,vxrxmORukEeUX8sN1Do67cKgLU04,2dZiMBwX_5Q,3,NaT
57,vxrxmORukEeUX8sN1Do67cKgLU04,TlB_eWDSMt4,5,NaT
58,vxrxmORukEeUX8sN1Do67cKgLU04,W6NZfCO5SIk,1,NaT
59,vxrxmORukEeUX8sN1Do67cKgLU04,kqtD5dpn9C8,4,NaT


In [7]:
print('Encoding data...')

# Complete userID refer to userid in user_df
actual_userId = np.unique(user_df.userid)
user_enc = LabelEncoder().fit(actual_userId)

# Complete videoID refer to videoid in video_df
video_userId = np.unique(video_df.videoId)
video_enc = LabelEncoder().fit(video_userId)

# encode the userID and videoId for all DataFrame
user_df['userId'] = user_enc.transform(user_df['userid'])
rating_df['userId'] = user_enc.transform(rating_df['userId'])
video_df['videoId'] = video_enc.transform(video_df['videoId'])
rating_df['videoId'] = video_enc.transform(rating_df['videoId'])

print('Encoding done!')

Encoding data...
Encoding done!


In [8]:
# Detect missing / unseen label

# x = np.unique([rating_df.userId])
# y = np.unique([user_df.userid])
# all = set(np.union1d(x, y))
# x[np.isin(x, y, invert=True)]

In [9]:
# if have any missing data/unseen data
# execute add the missing part manually

# seed_data = {
#     u'dobYear': 2000,
#     u'fname': 'gate',
#     u'gender': 'female',
#     u'satisfaction': 4.0,
#     u'userid': '',
#     u'email': 'gatetest2020@test.com',
#     u'age': 21,
#     u'dob': '2000 10 10',
#     u'lname': 'test2020',
#     u'role': 'admin',
#     u'videoswatched': 100,
# }

# db.collection(u'users').document('').set(seed_data)

In [10]:
final_rating_df = rating_df.drop(['whenRated'], axis=1)
final_rating_df.head()

,userId,videoId,value
0,0,0,3
1,0,3,5
2,0,6,3
3,0,8,3
4,0,11,4


In [11]:
# filter out rating row below 2 to remove low rating data
# suppose below 3, but due to insufficient data, thus loosen

train_rated = final_rating_df.copy()
train_rated_index = train_rated[train_rated['value'] < 2 ].index
train_rated.drop(train_rated_index, inplace=True)
train_rated

,userId,videoId,value
0,0,0,3
1,0,3,5
2,0,6,3
3,0,8,3
4,0,11,4
5,0,14,5
7,0,23,5
8,0,24,4
9,1,11,2
10,2,20,3


In [12]:
video_rated_by_user = final_rating_df.groupby('videoId')['userId'].count().reset_index(name='userIdCount')
video_rated_by_user = video_rated_by_user.set_index('videoId')
video_rated_by_user

,userIdCount
videoId,
0,12
1,2
2,1
3,3
4,1
5,1
6,3
7,1
8,8


In [13]:
# detect any videos that have no rated
def find_missing(lst):
    max_num = max(lst)
    min_num = min(lst)
    num_range = range(min_num, max_num+1)
    missing_number = list(set(num_range) - set(lst))
     
    return missing_number

# if have, find out the ID
index_list = video_rated_by_user.index.tolist()
what = find_missing(index_list)

# assign value '0' to respective item
for i in what:
  video_rated_by_user.loc[i] = [0]
video_rated_by_user

,userIdCount
videoId,
0,12
1,2
2,1
3,3
4,1
5,1
6,3
7,1
8,8


In [14]:
## No splitting due to insufficient data

# from sklearn.model_selection import train_test_split

# train, test = train_test_split(train_rated, test_size = 0.20, shuffle = False)

In [15]:
# # aggregate all the movieId that have been rated with above 3 by all user
# train_precision = train.copy().groupby('userId')['videoId'].agg(actual = lambda x: list(set(x)))
# test_precision = test.copy().groupby('userId')['videoId'].agg(actual = lambda x: list(set(x)))

In [16]:
## Need modification since the data used is different

"""
Precision computation
"""
def precision_per_user(user):
  # get the actual data
  check_a = test_precision.loc[user]
  result = float(len(np.intersect1d(check_a['actual'],check_a['predictions'])))
  # length of recommendation list
  L = min(len(check_a['actual']), len(check_a['predictions']))
  precision = (result/L)

  return precision

"""
Novelty computation
"""
def novelty_per_user(user):
  M = unique_user
  L = 10
  sum_novelty = 0
  video_list = test_precision['predictions'].loc[user]
  for m in video_list:
    rate_number = movie_rated_by_user.loc[m]
    if (int(rate_number) == 0):
      sum_novelty += 0
    else:
      sum_novelty += math.log2(M/rate_number)
  novelty = sum_novelty/len(video_list)
  return novelty

In [17]:
def check_precision(ind, a):

  high_item =  train_precision.loc[ind][0]
  correct_action = len(np.intersect1d(high_item,a))
  L = min(len(high_item),len(a))
  precision_value = correct_action/L

  return precision_value

def check_novelty(a):

  M = unique_user
  L = len(a)
  sum_novelty = 0

  for u in a:
    rate_number = video_rated_by_user.loc[u]
    if (int(rate_number) == 0):
      sum_novelty += 0
    else:
      sum_novelty += math.log2(M/rate_number)

  novelty_value = sum_novelty/len(a)
  return novelty_value

In [ ]:
class DQN:
    def __init__(self, state_dim, aciton_dim, ):
        self.state_dim  = state_dim
        self.action_dim = aciton_dim
        self.epsilon = 3.0
        self.eps_decay = 0.95
        self.min_epsilon = 0.5 
        self.model = self.build_dqn()
    
    def build_dqn(self):
        model = Sequential()
        model.add(keras.Input(shape=(self.state_size,)))
        model.add(keras.Dense(1024, activation=None,
                        kernel_initializer='he_uniform'))
        model.add(keras.Dense(2048, activation=None,
                        kernel_initializer='he_uniform'))
        model.add(keras.Dense(self.action_size, activation='relu',
                        kernel_initializer='he_uniform'))
        model.summary()
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model
    
    def predict(self, state):
        return self.model.predict(state)
    
    def get_action(self, state, action_num):
        state = np.reshape(state, [1, self.state_dim])
        self.epsilon *= self.eps_decay
        self.epsilon = max(self.epsilon, self.min_epsilon)
        q_value = self.predict(state)[0]
        if np.random.random() < self.epsilon:
            return random.randint(0, self.action_dim - 1, size = action_num)

        sorted_act = np.argsort(q_value, axis = 1)[:,::-1][:,:action_num]
        # rec_videos = [video_userId[i] for i in sorted_act]
        return sorted_act

    def train_Q_network(self, states, targets):
        self.model.fit(states, targets, epochs=1, verbose=0)

class MemoryBuffer:
    def __init__(self,mem_size=10000, batch_size):
        self.memory = deque(maxlen=mem_size)
        self.batch_size = batch_size
    
    def store(self, state, action, reward, next_state, done):
        self.memory.append([state, action, reward, next_state, done])
    
    def sample(self):
        sample = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, done = map(np.asarray, zip(*sample))
        states = np.array(states).reshape(self.batch_size, -1)
        next_states = np.array(next_states).reshape(self.batch_size, -1)
        return states, actions, rewards, next_states, done
    
    def size(self):
        return len(self.memory)

class Agent:
    def __init__(self, batch_size, state_size):
        self.state_dim = state_size
        self.action_dim = len(video_df)
        self.batch_size = batch_size
        self.q_model = DQN(self.state_dim, self.action_dim)
        self.target_model = DQN(self.state_dim, self.action_dim)
        self.target_update()
        self.gamma = 0.5

        self.buffer = MemoryBuffer(mem_size=10000, self.batch_size)

    def target_update(self):
        pred_weights = self.q_model.model.get_weights()
        self.target_model.model.set_weights(pred_weights)
    
    def replay(self):
        for step in range(10):
            states, actions, rewards, next_states, done = self.buffer.sample()
            targets = self.target_model.predict(states)
            next_q_values = self.target_model.predict(next_states).max(axis=1)
            targets[range(self.batch_size), actions] = rewards + (1-done) * next_q_values * self.gamma
            self.model.train_Q_network(states, targets)

In [ ]:
def compute_reward(action_list):
  rating = []
  for a in action_list:
    actual_videoId = video_enc.inverse_transform(a)
    avg_rating = video_df[video_df['videoId'] == actual_videoId].avgRating.values
    rating.append(avg_rating)
  return sum(rating)

In [ ]:
ax_epoch=100
train_data = np.array(train_rated)
batch_size = 10
recommend_length = 10
agent = Agent(batch_size, train_rated.shape[1])

for ep in range(max_epoch):
  done, total_reward = False, 0

  # range(0, len(train_data) - 1, batch_size)
  for i in range(0, len(train_data) - 2): 
    # index = min(i + batch_size, len(ww) - 1)  
    state = train_data[i: i+1]
    userid = state[0][0]
    action = self.model.get_action(state, len(recommend_length))
    # r_precision = check_precision(action, userid)
    # r_novelty = check_precision(action)
    reward = compute_reward(action)
    next_state = train_data[i+1: i+2]
    while not done:
      total_reward += reward
      agent.buffer.store(state, action, reward*0.01, next_state, next_state[0][0]!= state[0][0])
      if agent.buffer.size() >= batch_size:
        agent.replay()
        agent.target_update()
  print('EP{} EpisodeReward={}'.format(ep, total_reward))